In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim as gensimvis
from matplotlib import pyplot as plt
import ast

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['bigram_keyphrases'] = df['bigram_keyphrases'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else None)
    df['cleaned_bigrams'] = df['bigram_keyphrases'].apply(lambda x: [item[0].replace(' ', '_') for item in x] if x else [])
    return df

def filter_data(df):
    return df[(df['clean_text'].str.contains('spirituality', case=False)) & 
              (~df['clean_text'].str.contains('religion', case=False))]

def create_doc_term_matrix(data):
    dictionary = Dictionary(data['cleaned_bigrams'])
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in data['cleaned_bigrams']]
    return dictionary, doc_term_matrix

def train_lda_model(mallet_path, doc_term_matrix, dictionary, num_topics):
    lda = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=dictionary)
    os.makedirs('../models', exist_ok=True)
    lda.save(os.path.join('../models', 'lda_spirituality'))
    return lda

def convert_to_gensim_lda(lda_model_mallet):
    lda_model_gensim = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_mallet)
    return lda_model_gensim

def visualize_lda_model(lda_model_gensim, doc_term_matrix, dictionary, num_topics):
    vis_data = gensimvis.prepare(lda_model_gensim, doc_term_matrix, dictionary)
    html_dir = '../visualizations'
    os.makedirs(html_dir, exist_ok=True)
    html_filename = os.path.join(html_dir, f'lda_spirituality_{num_topics}_topics.html')
    pyLDAvis.save_html(vis_data, html_filename)


file_path = "/Users/shtosti/Dropbox/study/UZH/FW23/SMA/topic_modelling_DEPO/data/with_clean_keybert_bigrams.csv"
mallet_path = '/Users/shtosti/Dropbox/study/UZH/FW23/SMA/topic_modelling_DEPO/Mallet/mallet-2.0.8/bin/mallet'

num_topics = 10

# load data
df = load_data(file_path)

# filter data
filtered_df = filter_data(df)

# create vocabulary and doc-term matrix
dictionary, doc_term_matrix = create_doc_term_matrix(filtered_df)

# train
lda_model_mallet = train_lda_model(mallet_path, doc_term_matrix, dictionary, num_topics)

# convert to Gensim LDA model
lda_model_gensim = convert_to_gensim_lda(lda_model_mallet)

# visualize and save the HTML
visualize_lda_model(lda_model_gensim, doc_term_matrix, dictionary, num_topics)


/Users/shtosti/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/shtosti/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
Mallet LDA: 10 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 5
total tokens: 478859
<10> LL/token: -13.0878
<20> LL/token: -12.9238
<30> LL/token: -12.78684
<40> LL/token: -12.65505

0	5	spirituality spirituality_embeddedurl spiritual books_spirituality spiritual_warfare kindle_spirituality embeddedurl kindle love_spirituality spirituality_paranormal spiritual_books book_spirituality prayer ebook spiritual_book mooji_vedanta podcast spiritual_master psychic spiritual_heart 
1	5	